<a href="https://colab.research.google.com/github/suryateja1706/AIML-projects-and-labs/blob/main/Copy_of_AIML_III_Module_01_Lab_02_Machine_Learning_terms_and_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [57]:

import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
!python3 -m pip install certifi


rng = np.random.default_rng(seed=42)

import certifi
import os
os.environ['SSL_CERT_FILE'] = certifi.where()



In [58]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt


In [59]:
!pip install --upgrade scikit-learn

In [60]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived from the 1990 U.S. census, using one row per ce

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [61]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [62]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [63]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [64]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [65]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [66]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [67]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [68]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [69]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [70]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [71]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

In [75]:
#1
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Wine dataset
data = load_wine()
X, y = data.data, data.target

# Function to evaluate model performance with different validation set sizes
def evaluate_knn_with_different_splits(test_size):
    # Split the dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=42)

    # Train a 3-Nearest Neighbors classifier
    knn = KNeighborsClassifier(n_neighbors=3)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_val)

    # Calculate the accuracy of the classifier
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

# Evaluate with different validation set sizes
test_sizes = [0.1, 0.2, 0.3, 0.4, 0.5]
accuracies = [evaluate_knn_with_different_splits(test_size) for test_size in test_sizes]

# Print the accuracies for different validation set sizes
for test_size, accuracy in zip(test_sizes, accuracies):
    print(f"Validation set size: {test_size*100:.0f}%, Accuracy: {accuracy:.2f}")


Validation set size: 10%, Accuracy: 0.83
Validation set size: 20%, Accuracy: 0.81
Validation set size: 30%, Accuracy: 0.74
Validation set size: 40%, Accuracy: 0.67
Validation set size: 50%, Accuracy: 0.64


In [81]:
#2
# Define the range of train and validation set sizes
train_sizes = np.linspace(0.5, 0.7, 4)  # Reduced the maximum train_size to 0.7
val_sizes = np.linspace(0.1, 0.3, 4)

# Evaluate the model with different train and validation set sizes
accuracies = evaluate_knn_with_different_splits(train_sizes, val_sizes)

# Print the results
for train_size, val_size, test_accuracy in accuracies:
    print(f"Train size: {train_size:.1f}, Val size: {val_size:.1f}, Test accuracy: {test_accuracy:.2f}")

Train size: 0.5, Val size: 0.1, Test accuracy: 0.69
Train size: 0.5, Val size: 0.2, Test accuracy: 0.72
Train size: 0.5, Val size: 0.2, Test accuracy: 0.72
Train size: 0.5, Val size: 0.3, Test accuracy: 0.75
Train size: 0.6, Val size: 0.1, Test accuracy: 0.69
Train size: 0.6, Val size: 0.2, Test accuracy: 0.72
Train size: 0.6, Val size: 0.2, Test accuracy: 0.72
Train size: 0.6, Val size: 0.3, Test accuracy: 0.72
Train size: 0.6, Val size: 0.1, Test accuracy: 0.69
Train size: 0.6, Val size: 0.2, Test accuracy: 0.69
Train size: 0.6, Val size: 0.2, Test accuracy: 0.72
Train size: 0.6, Val size: 0.3, Test accuracy: 0.72
Train size: 0.7, Val size: 0.1, Test accuracy: 0.69
Train size: 0.7, Val size: 0.2, Test accuracy: 0.72
Train size: 0.7, Val size: 0.2, Test accuracy: 0.72
Train size: 0.7, Val size: 0.3, Test accuracy: 0.69


In [82]:

#3
from sklearn.model_selection import train_test_split

# Example dataset size
total_samples = 1000

# Calculate sizes for training and validation sets
train_size = 0.7  # 70% for training
val_size = 0.3    # 30% for validation

# Calculate actual number of samples
train_samples = int(train_size * total_samples)
val_samples = int(val_size * total_samples)

print(f"Training samples: {train_samples}")
print(f"Validation samples: {val_samples}")


Training samples: 700
Validation samples: 300


In [74]:
#excercise
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load the Wine dataset
data = load_wine()
X, y = data.data, data.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a 1-Nearest Neighbor classifier
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train, y_train)
y_pred_1nn = knn1.predict(X_test)

# Train a 3-Nearest Neighbors classifier
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(X_train, y_train)
y_pred_3nn = knn3.predict(X_test)

# Calculate the accuracy of the 1-NN classifier
accuracy_1nn = accuracy_score(y_test, y_pred_1nn)

# Calculate the accuracy of the 3-NN classifier
accuracy_3nn = accuracy_score(y_test, y_pred_3nn)

# Print the accuracy of both classifiers
print(f"Accuracy of 1-NN classifier: {accuracy_1nn:.2f}")
print(f"Accuracy of 3-NN classifier: {accuracy_3nn:.2f}")


Accuracy of 1-NN classifier: 0.78
Accuracy of 3-NN classifier: 0.81


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [83]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [84]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


In [85]:

#1
from sklearn.datasets import load_wine
from sklearn.model_selection import cross_val_score, KFold
from sklearn.neighbors import KNeighborsClassifier

# Load the Wine dataset
data = load_wine()
X, y = data.data, data.target

# Initialize K-Nearest Neighbors classifier
knn = KNeighborsClassifier(n_neighbors=3)

# Define the number of folds for cross-validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Perform cross-validation
cv_scores = cross_val_score(knn, X, y, cv=kf)

# Print the cross-validation scores
print(f"Cross-validation scores: {cv_scores}")
print(f"Average validation accuracy: {cv_scores.mean():.2f} +/- {cv_scores.std():.2f}")


Cross-validation scores: [0.80555556 0.61111111 0.63888889 0.65714286 0.82857143]
Average validation accuracy: 0.71 +/- 0.09


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

In [16]:
#exercise
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

# Load the dataset
dataset = fetch_california_housing()
X, y = dataset.data, dataset.target

# Convert the regression problem into a binary classification problem
# We'll classify the data into two classes: above median value and below/equal median value
median_value = np.median(y)
y_class = (y > median_value).astype(int)

# Create a pipeline with scaling and classifier
def get_knn_classifier(n_neighbors):
    return make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=n_neighbors))

# Define different configurations for cross-validation and train/test splits
cv_folds = [5, 10]
test_sizes = [0.2, 0.3]

# Initialize results dictionary
results = {'1-NN': {}, '3-NN': {}}

# Perform cross-validation
for cv in cv_folds:
    for n_neighbors, classifier_name in zip([1, 3], ['1-NN', '3-NN']):
        classifier = get_knn_classifier(n_neighbors)
        scores = cross_val_score(classifier, X, y_class, cv=cv, scoring='accuracy')
        results[classifier_name][f'{cv}-fold CV'] = scores.mean()

# Evaluate with different train/test splits
for test_size in test_sizes:
    for n_neighbors, classifier_name in zip([1, 3], ['1-NN', '3-NN']):
        X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=test_size, random_state=42)
        classifier = get_knn_classifier(n_neighbors)
        classifier.fit(X_train, y_train)
        accuracy = classifier.score(X_test, y_test)
        results[classifier_name][f'Train/Test Split {int((1-test_size)*100)}/{int(test_size*100)}'] = accuracy

# Print results
for classifier_name, metrics in results.items():
    print(f"Results for {classifier_name}:")
    for metric, score in metrics.items():
        print(f"  {metric}: {score:.4f}")


Results for 1-NN:
  5-fold CV: 0.7417
  10-fold CV: 0.7257
  Train/Test Split 80/20: 0.8023
  Train/Test Split 70/30: 0.8064
Results for 3-NN:
  5-fold CV: 0.7586
  10-fold CV: 0.7412
  Train/Test Split 80/20: 0.8232
  Train/Test Split 70/30: 0.8272
